In [6]:
import argparse
import errno
import os
import pychord
import time
import traceback

from midiutil import MIDIFile
from mingus.core.progressions import to_chords, determine
import mingus.core.notes as notes

In [18]:
track = 0
channel = 0
ttime = 0
duration = 1 # In beats
tempo = 80  # In BPM
volume = 100  # 0-127, as per the MIDI standard
bar = 0
humanize_interval = 0.0
directory = False
num_notes = 99
offset = 0.0
key = 'C'
octaves = '4'
root_lowest = False
bassline = False
pattern = False

# Could be interesting to do multiple parts at once.
midi = MIDIFile(1)
midi.addTempo(track, ttime, tempo)

has_number = False

In [26]:
progression_chords = []
progression = "I,V,vi,IV".split(",")
og_progression = progression

In [24]:
to_chords("vi", "C")

[['A', 'C', 'E']]

In [27]:
for chord in progression:

    # This is for # 'I', 'VI', etc
    progression_chord = to_chords(chord, key)
    if progression_chord != []:
        has_number = True
    print(progression_chord)
    # This is for 'C', 'Am', etc.
    if progression_chord == []:
        try:
            progression_chord = [pychord.Chord(chord).components()]
        except Exception:
            # This is an 'X' input
            progression_chord = [None]

    chord_info = {}
    chord_info['notes'] = progression_chord[0]
    if has_number:
        chord_info['number'] = chord
    else:
        chord_info['name'] = chord

    if progression_chord[0]:
        chord_info['root'] = progression_chord[0][0]
    else:
        chord_info['root'] = None
    progression_chords.append(chord_info)

[['C', 'E', 'G']]
[['G', 'B', 'D']]
[['A', 'C', 'E']]
[['F', 'A', 'C']]


In [22]:
progression

'I,V,vi,IV'

In [21]:
print(progression_chords)

[{'notes': ['C', 'E', 'G'], 'number': 'I', 'root': 'C'}, {'notes': None, 'number': ',', 'root': None}, {'notes': ['G', 'B', 'D'], 'number': 'V', 'root': 'G'}, {'notes': None, 'number': ',', 'root': None}, {'notes': ['G', 'B', 'D'], 'number': 'v', 'root': 'G'}, {'notes': ['C', 'E', 'G'], 'number': 'i', 'root': 'C'}, {'notes': None, 'number': ',', 'root': None}, {'notes': ['C', 'E', 'G'], 'number': 'I', 'root': 'C'}, {'notes': ['G', 'B', 'D'], 'number': 'V', 'root': 'G'}]


In [28]:

# For each input..
previous_pitches = []
for chord_index, chord_info in enumerate(progression_chords):

    # Unpack object
    chord = chord_info['notes']
    # NO_OP
    if chord == None:
        bar=bar+1
        continue
    root = chord_info['root']
    root_pitch = pychord.utils.note_to_val(notes.int_to_note(notes.note_to_int(root)))

    # Reset internals
    humanize_amount = humanize_interval
    pitches = []
    all_new_pitches = []

    # Turns out this algorithm was already written in the 1800s!
    # https://en.wikipedia.org/wiki/Voice_leading#Common-practice_conventions_and_pedagogy

    # a) When a chord contains one or more notes that will be reused in the chords immediately following, then these notes should remain, that is retained in the respective parts.
    # b) The parts which do not remain, follow the law of the shortest way (Gesetze des nachsten Weges), that is that each such part names the note of the following chord closest to itself if no forbidden succession XXX GOOD NAME FOR A BAND XXX arises from this.
    # c) If no note at all is present in a chord which can be reused in the chord immediately following, one must apply contrary motion according to the law of the shortest way, that is, if the root progresses upwards, the accompanying parts must move downwards, or inversely, if the root progresses downwards, the other parts move upwards and, in both cases, to the note of the following chord closest to them.
    root = None
    for i, note in enumerate(chord):

        # Sanitize notes
        sanitized_notes = notes.int_to_note(notes.note_to_int(note))
        pitch = pychord.utils.note_to_val(sanitized_notes)

        if i == 0:
            root = pitch

        if root:
            if root_lowest and pitch < root: # or chord_index is 0:
                pitch = pitch + 12 # Start with the root lowest

        all_new_pitches.append(pitch)

        # Reuse notes
        if pitch in previous_pitches:
            pitches.append(pitch)

    no_melodic_fluency = False # XXX: vargify
    if previous_pitches == [] or all_new_pitches == [] or pitches == [] or no_melodic_fluency:
        pitches = all_new_pitches
    else:
        # Detect the root direction
        root_upwards = None
        if pitches[0] >= all_new_pitches[0]:
            root_upwards = True
        else:
            root_upwards = False

        # Move the shortest distance
        if pitches != []:
            new_remaining_pitches = list(all_new_pitches)
            old_remaining_pitches = list(previous_pitches)
            for i, new_pitch in enumerate(all_new_pitches):
                # We're already there
                if new_pitch in pitches:
                    new_remaining_pitches.remove(new_pitch)
                    old_remaining_pitches.remove(new_pitch)
                    continue

            # Okay, so need to find the overall shortest distance from the remaining pitches - including their permutations!
            while len(new_remaining_pitches) > 0:
                nearest_distance = 9999
                previous_index = None
                new_index = None
                pitch_to_add = None
                for i, pitch in enumerate(new_remaining_pitches):
                    # XXX: DRY

                    # The Pitch
                    pitch_to_test = pitch
                    nearest = min(old_remaining_pitches, key=lambda x:abs(x-pitch_to_test))
                    old_nearest_index = old_remaining_pitches.index(nearest)
                    if nearest < nearest_distance:
                        nearest_distance = nearest
                        previous_index = old_nearest_index
                        new_index = i
                        pitch_to_add = pitch_to_test

                    # +12
                    pitch_to_test = pitch + 12
                    nearest = min(old_remaining_pitches, key=lambda x:abs(x-pitch_to_test))
                    old_nearest_index = old_remaining_pitches.index(nearest)
                    if nearest < nearest_distance:
                        nearest_distance = nearest
                        previous_index = old_nearest_index
                        new_index = i
                        pitch_to_add = pitch_to_test

                    # -12
                    pitch_to_test = pitch - 12
                    nearest = min(old_remaining_pitches, key=lambda x:abs(x-pitch_to_test))
                    old_nearest_index = old_remaining_pitches.index(nearest)
                    if nearest < nearest_distance:
                        nearest_distance = nearest
                        previous_index = old_nearest_index
                        new_index = i
                        pitch_to_add = pitch_to_test

                # Before we add it - just make sure that there isn't a better place for it.
                pitches.append(pitch_to_add)
                del old_remaining_pitches[previous_index]
                del new_remaining_pitches[new_index]

                # This is for the C E7 type scenario
                if len(old_remaining_pitches) == 0:
                    for x, extra_pitch in enumerate(new_remaining_pitches):
                        pitches.append(extra_pitch)
                        del new_remaining_pitches[x]

            # Final check - can the highest and lowest be safely folded inside?
            max_pitch = max(pitches)
            min_pitch = min(pitches)
            index_max = pitches.index(max_pitch)
            folded_max = max_pitch - 12
            if (folded_max > min_pitch) and (folded_max not in pitches):
                pitches[index_max] = folded_max

            max_pitch = max(pitches)
            min_pitch = min(pitches)
            index_min = pitches.index(min_pitch)

            folded_min = min_pitch + 12
            if (folded_min < max_pitch) and (folded_min not in pitches):
                pitches[index_min] = folded_min

            # Make sure the average can't be improved
            # XXX: DRY
            if len(previous_pitches) != 0:
                previous_average = sum(previous_pitches) / len(previous_pitches)

                # Max
                max_pitch = max(pitches)
                min_pitch = min(pitches)
                index_max = pitches.index(max_pitch)
                folded_max = max_pitch - 12

                current_average = sum(pitches) / len(pitches)
                hypothetical_pitches = list(pitches)
                hypothetical_pitches[index_max] = folded_max
                hypothetical_average = sum(hypothetical_pitches) / len(hypothetical_pitches)
                if abs(previous_average-hypothetical_average) <= abs(previous_average-current_average):
                    pitches[index_max] = folded_max
                # Min
                max_pitch = max(pitches)
                min_pitch = min(pitches)
                index_min = pitches.index(min_pitch)
                folded_min = min_pitch + 12

                current_average = sum(pitches) / len(pitches)
                hypothetical_pitches = list(pitches)
                hypothetical_pitches[index_min] = folded_min
                hypothetical_average = sum(hypothetical_pitches) / len(hypothetical_pitches)
                if abs(previous_average-hypothetical_average) <= abs(previous_average-current_average):
                    pitches[index_min] = folded_min

        # Apply contrary motion
        else:
            print ("Applying contrary motion!")
            for i, new_pitch in enumerate(all_new_pitches):
                if i == 0:
                    pitches.append(new_pitch)
                    continue

                # Root upwards, the rest move down.
                if root_upwards:
                    if new_pitch < previous_pitches[i]:
                        pitches.append(new_pitch)
                    else:
                        pitches.append(new_pitch - 12)
                else:
                    if new_pitch > previous_pitches[i]:
                        pitches.append(new_pitch)
                    else:
                        pitches.append(new_pitch + 12)

    # Bassline
    if bassline:
        pitches.append(root_pitch - 24)

    # Melody

    # Octave is a simple MIDI offset counter
    for octave in octaves:
        for note in pitches:
            pitch = int(note) + (int(octave.strip()) * 12)

            # Don't humanize bassline note
            if bassline and (pitches.index(note) == len(pitches) -1):
                midi_time = offset + bar
            else:
                midi_time = offset + bar + humanize_amount

            # Write the note
            midi.addNote(
                track=track,
                channel=channel,
                pitch=pitch,
                time=midi_time,
                duration=duration,
                volume=volume
            )

        humanize_amount = humanize_amount + humanize_interval
        if i + 1 >= num_notes:
            break
    bar = bar + 1
    previous_pitches = pitches

In [29]:
with open("major-scale.mid", "wb") as output_file:
    midi.writeFile(output_file)